In [1]:
import pandas as pd
import pickle
from natsort import natsorted

In [2]:
df = pd.read_csv("../data/dataset-02.csv")

In [3]:
df.head()

,time,Unnamed: 0,year,month,week,province,city,store,category,department,class,vendor,size,totalQuantity,totalSales,Holiday
0,2012-01-08,0,2012,1,1,province3,city31,store40,category1,department1,class7,vendor2,size26,1.0,424.971081,True
1,2012-01-08,1,2012,1,1,province2,city15,store33,category1,department1,class7,vendor2,size29,-1.0,-424.971081,True
2,2012-01-08,3,2012,1,1,province1,city18,store51,category1,department1,class7,vendor2,size29,1.0,424.971081,True
3,2012-01-08,4,2012,1,1,province3,city31,store62,category1,department1,class7,vendor2,size29,1.0,424.971081,True
4,2012-01-08,5,2012,1,1,province3,city31,store40,category1,department1,class7,vendor2,size33,1.0,424.971081,True


In [4]:
df = df.drop(columns="Unnamed: 0")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2348930 entries, 0 to 2348929
Data columns (total 15 columns):
 #   Column         Dtype  
---  ------         -----  
 0   time           object 
 1   year           int64  
 2   month          int64  
 3   week           int64  
 4   province       object 
 5   city           object 
 6   store          object 
 7   category       object 
 8   department     object 
 9   class          object 
 10  vendor         object 
 11  size           object 
 12  totalQuantity  float64
 13  totalSales     float64
 14  Holiday        bool   
dtypes: bool(1), float64(2), int64(3), object(9)
memory usage: 253.1+ MB


In [6]:
#unique values in each column
print("Unique values in each column")
print("----------------------------")
for col in list(df):
    print(col,":",len(df[col].unique()))

Unique values in each column
----------------------------
time : 313
year : 7
month : 12
week : 52
province : 3
city : 44
store : 57
category : 2
department : 14
class : 44
vendor : 95
size : 40
totalQuantity : 81
totalSales : 42510
Holiday : 2


print all possible combination between space and product hierarchy

In [7]:
space = ["province", "city","store"]
prod = ["category","department","class"]
time = ["year", "weekly","monthly","quarterly"]
total = 0
all_cross_terms = []
cross_groups = []
print("Count of all cross-combination between 2 hierarchy")
print("--------------------------------------------------")
for x in space:
    for y in prod:
        grp_obj = df.groupby([x,y])
        grps = grp_obj.ngroups
        all_cross_terms.extend(grp_obj.groups.keys())
        cross_groups.append(dict(tuple(grp_obj)))
        total = total + grps
        print(x,"VS",y,":", grps)
print("Total cross terms:", total)
print(total == len(all_cross_terms))
print(len(cross_groups))

cross_dict = {}
[cross_dict.update(dic) for dic in cross_groups];
print('Total cross dicts:', len(cross_dict))

Count of all cross-combination between 2 hierarchy
--------------------------------------------------
province VS category : 6
province VS department : 39
province VS class : 127
city VS category : 88
city VS department : 505
city VS class : 1740
store VS category : 114
store VS department : 641
store VS class : 2217
Total cross terms: 5477
True
9
Total cross dicts: 5477


Get all individual combinations

In [8]:
groups = []
group_terms = []
for x in space:
    grp_obj = df.groupby(x)
    groups.append(dict(tuple(grp_obj)))
    group_terms.extend(grp_obj.groups.keys())
print('Completed:',space)

for y in prod:
    grp_obj = df.groupby(y)
    groups.append(dict(tuple(grp_obj)))
    group_terms.extend(grp_obj.groups.keys())
print('Completed:',prod)

group_dict = {}
[group_dict.update(dic) for dic in groups];
print('Total group dicts:',len(group_dict))


Completed: ['province', 'city', 'store']
Completed: ['category', 'department', 'class']
Total group dicts: 164


Merge individual combinations and cross terms

In [9]:
all_terms = group_terms.copy()
all_terms.extend(all_cross_terms)
print('Total terms:',len(all_terms))

Total terms: 5641


In [10]:
complete_dict = group_dict
complete_dict.update(cross_dict)
print('Total groupings:',len(complete_dict))
print(len(complete_dict) == len(all_terms))


Total groupings: 5641
True


In [12]:
len(group_terms), len(all_cross_terms), len(all_terms)

(164, 5477, 5641)

count data points in each cross-combinations

In [ ]:
print("SPACE      PRODUCT        COUNT")
print("-------------------------------")
for x in space:
    for y in prod:
        print(df.groupby([x,y]).size())

find combinations that do not exist

In [ ]:
grp = list(df.groupby(["province","department"]).groups.keys())
natsorted(grp)

we can see above that province1+department14, province3+department6, province3+department14 do not exist.

In [ ]:
# save the cross_terms to a file. Need to run only once
with open("../data/cross_terms.pickle", "wb") as f:
    pickle.dump(all_cross_terms, f)

In [ ]:
# save the all terms to a file. Need to run only once
with open("../data/all_terms.pickle", "wb") as f:
    pickle.dump(all_terms, f)